In [24]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# 2019, 2020, 2021, 2022 기업과 코스피 등락률 csv 파일 병합 및 로지스틱 회귀 함수
def for_read_csvs(name):
    # 년도별 기업과 코스피 등락률 csv 파일 병합
    df1 = pd.read_csv('./csv/2018_'+name+'_코스피등락률.csv', encoding='CP949')
    df2 = pd.read_csv('./csv/2019_'+name+'_코스피등락률.csv', encoding='CP949')
    df3 = pd.read_csv('./csv/2020_'+name+'_코스피등락률.csv', encoding='CP949')
    df4 = pd.read_csv('./csv/2021_'+name+'_코스피등락률.csv', encoding='CP949')
    
    # 테스트 셋용 3년치 데이터 프레임 병합
    df_3yrs = pd.concat([df1, df2, df3])
    df_3yrs.reset_index(drop=True, inplace=True)
    
    # 데이터 셋 분리
    X_train = df_3yrs[["기업 등락률", "코스피 등락률"]]
    X_test = df4[["기업 등락률", "코스피 등락률"]]
    y_train = df_3yrs["베타"]
    y_test = df4["베타"]
    
    # Standard Scaling
    sscaler = StandardScaler()

    X_train = sscaler.fit_transform(X_train)
    X_test = sscaler.fit_transform(X_test)
    
    # 로지스틱 회귀
    model = LogisticRegression()
    results = model.fit(X_train, y_train)
    
    # 모델의 예측 학습
    model_pred = model.predict(X_test)

    print("roc_auc: ", roc_auc_score(y_test, model_pred))
    print("트레이닝 데이터 모델 정확도: ", model.score(X_train, y_train))
    print("테스트 데이터 모델 정확도: ", model.score(X_test, y_test))

In [25]:
for_read_csvs("동양생명")

ValueError: Unknown label type: 'continuous'

In [18]:
model.coef_

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [ ]:
from sklearn.model_selection import GridSearchCV

# 주요 파라메터 (C)
# C 값 (기본값 = 1)
# C 값이 작으면 Penalty 강해짐 (단순 모델)
# C 값이 크면 Penalty 약해짐 (정규화 없어짐)
# 보통 로그스케일로 지정(10배씩) = 0.01, 0.1, 1, 10

# penalty
# L2: Ridge, 일반적으로 사용 (default)
# L1: LASSO, 변수가 많아서 줄여야할 때 사용, 모델의 단순화 및 해석에 용이
params={'penalty':['l2', 'l1'],
       'C':[0.001, 0.01, 0.1, 1, 10, 100]}

grid_clf = GridSearchCV(model, param_grid=params, scoring='accuracy', cv=5)

grid_clf.fit(X_train, y_train)
print('최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, 
                                                                    grid_clf.best_score_))

In [ ]:
from sklearn import metrics 
import matplotlib.pyplot as plt

# TPR(재현율) = 민감도
# TNR = 특이성 
metrics.plot_roc_curve(grid_clf, X_test, y_test)

plt.plot([0,1], [0,1], 'k--')
plt.show()

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

dpredictions = grid_clf.predict(X_test)

precision = precision_score(y_test, dpredictions)
accuracy = accuracy_score(y_test, dpredictions)
recall = recall_score(y_test, dpredictions)

print('테스트 세트에서의 LogisticRegression 정밀도 : {0:.4f}'.format(precision))
print('테스트 세트에서의 LogisticRegression 정확도 : {0:.4f}'.format(accuracy))
print('테스트 세트에서의 LogisticRegression 재현율 : {0:.4f}'.format(recall))